
# Physician Conversion Model

This is a modeling pipeline for predicting whether a physician will convert to a new platform. The pipeline consists of the following steps:

1. Split the data into train, validation, and inference sets.
2. Train a variety of models without hyperparameter tuning (vanilla models).
3. Select one of the vanilla models and tune its hyperparameters.
4. Evaluate the model on the inference set.

## Step 1: Split Data into Train, Validation, and Inference Sets

The data was split into 70% train, 20% validation, and 10% inference sets. This ensures that we have enough data to train the model, validate the model, and evaluate the model on unseen data.

## Step 2: Train Vanilla Models

A variety of vanilla models were trained, including logistic regression, decision trees, and random forests. These models were trained without hyperparameter tuning.

## Step 3: Select a Model

One of the vanilla models (----) was selected for hyperparameter tuning. The hyperparameters that were tuned include the learning rate and the regularization strength.

## Step 4: Evaluate the Model

The tuned model was evaluated on the validation set. The model achieved a high F1-score, indicating that it is able to predict whether a physician will convert to a new platform with a high degree of accuracy.

## Conclusion

The modeling pipeline described in this document was able to achieve a high F1-score on the validation set. This suggests that the model is able to predict whether a physician will convert to a new platform with a high degree of accuracy.

## Next Step

The final/selected model will be used in Inference Pipeline to do predition on Inference set

### Import Libraries and Model Input Dataset

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#Visual Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Importing necessary libraries for encoding
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Importing necessary library for scaling
from sklearn.preprocessing import StandardScaler

# Importing necessary library for train-test split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Importing necessary libraries for model development and evaluation
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
import xgboost as xgb
import lightgbm as lgb

# Hyperparameter Tuning
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
import os

### Loading Data

In [39]:
from sagemaker.feature_store.feature_group import FeatureGroup
feature_group_name = "physician-conversion-feature-group-30-14-22-46"

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Load the feature group
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

In [43]:
query = feature_group.athena_query()
database = query.database
table_name = query.table_name
print(database)
print('')
print(table_name)

sagemaker_featurestore

physician_conversion_feature_group_30_14_22_46_1714486966


In [49]:
bucket = "sagemaker-experiment-hs/"
prefix = "Feature-store-trial"

In [51]:
query_string = 'SELECT * FROM "{}"."{}"'.format(database, table_name)
query.run(
    query_string=query_string,
    output_location='s3://sagemaker-experiment-hs/Feature-store-trial/'
)
query.wait()
dataset = query.as_dataframe()


In [52]:
dataset.head()

,npi_id,hcp_id,target,age,year_of_experience,number_of_rx,rx_last_1_month,rx_last_3_month,rx_last_6_month,rx_last_12_month,...,specialty_oncology,specialty_pediatric,specialty_uro-oncology,hco_affiliation_type_contract,hco_affiliation_type_employment,hco_affiliation_type_referral,eventtime,write_time,api_invocation_time,is_deleted
0,9875108,HCP_14,1,66,38,1100,1740,2884,3396,5605,...,False,False,False,False,False,True,1.714487e+09,2024-04-30 14:28:40.579,2024-04-30 14:23:28.000,False
1,4936853,HCP_45,0,68,13,833,1569,2861,3119,3437,...,False,True,False,False,True,False,1.714487e+09,2024-04-30 14:28:40.579,2024-04-30 14:23:29.000,False
2,6801445,HCP_62,0,41,60,490,932,1118,1700,2689,...,False,False,False,False,False,True,1.714487e+09,2024-04-30 14:28:40.579,2024-04-30 14:23:29.000,False
3,8159501,HCP_116,0,59,60,1100,2082,3317,5516,9072,...,False,True,False,True,False,False,1.714487e+09,2024-04-30 14:28:40.579,2024-04-30 14:23:30.000,False
4,3696556,HCP_198,0,72,10,243,246,256,342,345,...,False,False,True,True,False,False,1.714487e+09,2024-04-30 14:28:40.579,2024-04-30 14:23:31.000,False


### Load model features from s3

In [55]:
import pickle

# Initialize the S3 client
s3 = boto3.client('s3')

# Specify the bucket name and the key (file path) of the pickle file in S3
bucket_name = 'sagemaker-experiment-hs'
key = 'processed_output//model_train_col_list.pkl'

# Load the pickle file from S3
response = s3.get_object(Bucket=bucket_name, Key=key)

# Read the content of the pickle file
pickle_bytes = response['Body'].read()

# Load the pickle bytes into a Python object
model_var_list = pickle.loads(pickle_bytes)

# Now you can use the loaded_object as needed
print(model_var_list)


['NPI_ID', 'HCP_ID', 'TARGET', 'Age', 'Year_of_Experience', 'Number_of_Rx', 'Rx_last_1_Month', 'Rx_last_3_Month', 'Rx_last_6_Month', 'Rx_last_12_Month', 'Number_of_Px', 'Px_last_3_Month', 'Claims_last_1_Month', 'Claims_last_3_Month', 'Claims_last_6_Month', 'Claims_last_12_Month', 'Procedures_chemo_last_1_month', 'Procedures_chemo_last_3_month', 'Procedures_chemo_last_6_month', 'Procedures_chemo_last_12_month', 'Procedures_radio_last_6_month', 'Procedures_radio_last_12_month', 'Procedures_Biopsy_last_6_month', 'Procedures_Biopsy_last_12_month', 'Promotional_doximity', 'Promotional_medscape', 'Promotional_medscape_last_1_month', 'Promotional_medscape_last_3_month', 'Promotional_medscape_last_6_month', 'Promotional_medscape_last_12_month', 'VRC_visit_last_6_month', 'VRC_visit_last_12_month']


In [58]:
model_var_list = [item.lower() for item in model_var_list]

key_cols = ['npi_id', 'hcp_id']

for item in key_cols:
    if item in model_var_list:
        model_var_list.remove(item)
print(model_var_list)

['target', 'age', 'year_of_experience', 'number_of_rx', 'rx_last_1_month', 'rx_last_3_month', 'rx_last_6_month', 'rx_last_12_month', 'number_of_px', 'px_last_3_month', 'claims_last_1_month', 'claims_last_3_month', 'claims_last_6_month', 'claims_last_12_month', 'procedures_chemo_last_1_month', 'procedures_chemo_last_3_month', 'procedures_chemo_last_6_month', 'procedures_chemo_last_12_month', 'procedures_radio_last_6_month', 'procedures_radio_last_12_month', 'procedures_biopsy_last_6_month', 'procedures_biopsy_last_12_month', 'promotional_doximity', 'promotional_medscape', 'promotional_medscape_last_1_month', 'promotional_medscape_last_3_month', 'promotional_medscape_last_6_month', 'promotional_medscape_last_12_month', 'vrc_visit_last_6_month', 'vrc_visit_last_12_month']


In [62]:
bucket = "sagemaker-experiment-hs"
prefix = "processed_output"

In [63]:
# Select useful columns for training with target column as the first.
# Initialize the S3 client
s3_client = boto3.client('s3')
dataset = dataset[model_var_list]

# Write to csv in S3 without headers and index column.
dataset.to_csv("dataset.csv", header=False, index=False)
s3_client.upload_file("dataset.csv", bucket, prefix + "/training_input/dataset.csv")
dataset_uri_prefix = "s3://" + bucket + "/" + prefix + "/training_input/"

dataset

,target,age,year_of_experience,number_of_rx,rx_last_1_month,rx_last_3_month,rx_last_6_month,rx_last_12_month,number_of_px,px_last_3_month,...,procedures_biopsy_last_6_month,procedures_biopsy_last_12_month,promotional_doximity,promotional_medscape,promotional_medscape_last_1_month,promotional_medscape_last_3_month,promotional_medscape_last_6_month,promotional_medscape_last_12_month,vrc_visit_last_6_month,vrc_visit_last_12_month
0,1,66,38,1100,1740,2884,3396,5605,132,323,...,264,383,10,8,9,13,22,40,7,12
1,0,68,13,833,1569,2861,3119,3437,159,280,...,82,135,8,8,15,27,34,43,8,15
2,0,41,60,490,932,1118,1700,2689,220,461,...,185,366,5,10,13,18,33,38,5,10
3,0,59,60,1100,2082,3317,5516,9072,273,608,...,200,286,10,9,17,21,26,31,11,21
4,0,72,10,243,246,256,342,345,143,222,...,120,215,6,10,12,22,24,42,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,88,60,1100,1661,2416,4141,6776,169,360,...,127,236,6,6,8,14,20,26,8,12
4996,0,48,48,1086,1394,1540,2660,4127,115,266,...,125,132,6,6,11,14,18,33,8,14
4997,0,41,31,1100,1524,1546,3029,4731,235,906,...,156,240,6,9,11,19,28,50,9,12
4998,0,32,20,166,270,495,663,1151,130,376,...,75,106,8,5,9,12,17,20,5,8
